In [106]:
%matplotlib inline


# Bird Visulaization project
This Jupyter notebook looks serves for exploration capabilioties regarding the visualizations that we have in mind


### Imports and Load Data

This cell will contain all the imports we need to keep it stuctured when running the project

The line that imports df_cleaned_result has the logic for the df_cleaning.py


In [107]:
import sys
import os
import dash
from dash import html, dcc, Input, Output #Replaces dash_html_components as of Dash 2.0
import dash_bootstrap_components as dbc # Highly recommended for easy styling
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.abspath(".."))  # go up from notebooks/ to the project root

from data.df_cleaning import df_cleaned_result

df = df_cleaned_result.copy()



# Exploration of the Dataset

This next cell explores the final dataframe 

In [108]:
print(len(df_cleaned_result))
print("Dataset shape:", df_cleaned_result.shape)
print("\nColumn names and types:")
print(df_cleaned_result.dtypes)
print("\nFirst few rows:")
print(df_cleaned_result.head())
print("\nBasic statistics:")
print(df_cleaned_result.describe())
print("\nMissing values:")
print(df_cleaned_result.isnull().sum())

if 'species' in df_cleaned_result.columns or 'bird_name' in df_cleaned_result.columns:
    print("\nUnique species count:", df_cleaned_result['species'].nunique())

9999
Dataset shape: (9999, 27)

Column names and types:
Bird_ID                   object
Species                   object
Region                    object
Habitat                   object
Weather_Condition         object
Migration_Reason          object
Start_Latitude           float64
Start_Longitude          float64
End_Latitude             float64
End_Longitude            float64
Flight_Distance_km       float64
Flight_Duration_hours    float64
Average_Speed_kmph       float64
Max_Altitude_m             int64
Min_Altitude_m             int64
Temperature_C            float64
Wind_Speed_kmph          float64
Migration_Start_Month     object
Migration_End_Month       object
Rest_Stops                 int64
Migrated_in_Flock         object
Flock_Size                 int64
Migration_Interrupted     object
Interrupted_Reason        object
Migration_Success         object
Observation_Counts         int64
Origin                    object
dtype: object

First few rows:
  Bird_ID  Species    

# Summary Statistics


In [109]:
# Month ordering
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


total_observations = len(df)
success_rate = (df['Migration_Success'].str.lower() == 'successful').sum() / total_observations
percent_successful_migration = f"{success_rate * 100:.1f}%"
num_species = df['Species'].nunique()
average_distance = df['Flight_Distance_km'].mean()
avg_distance_formatted = f"{average_distance:,.0f} km"
average_duration = df['Flight_Duration_hours'].mean()
avg_duration_formatted = f"{average_duration:,.0f} hours"
max_altitude = df['Max_Altitude_m'].max()
max_altitude_formatted = f"{max_altitude:,.0f} m"
longest_flight = df['Flight_Distance_km'].max()
longest_flight_formatted = f"{longest_flight:,.0f} km"

summary_stats = {
    'Total Observations': total_observations,
    '% of Successful Migration': percent_successful_migration,
    'Maximal Altitude': max_altitude_formatted,
    'Average Flight Duration': avg_duration_formatted,
    'Number Of Species': num_species,
    'Average Flight Distance': avg_distance_formatted,
    'Longest Flight': longest_flight_formatted
}

### Helper Functions for the app


In [110]:
def make_kpi_card(title, value, icon_class):
    """Creates a stylized KPI card component."""
    return dbc.Card(
        dbc.CardBody([
            html.Div(className=icon_class, style={'fontSize': '2.5rem', 'color': "#0259b1ff"}),
            html.H4(value, className="card-title", style={'color': "#303030"}),
            html.P(title, className="card-text text-muted"),
        ], style={'textAlign': 'center', 'padding': '1.5rem'}),
        className="mb-4 shadow-sm h-100",
    )

def filter_data(species_list, region_list, month_list, temp_range, dist_range, alt_range):
    """Filter dataframe based on selections"""
    filtered = df.copy()

    if species_list and len(species_list) > 0:
        filtered = filtered[filtered['Species'].isin(species_list)]
    if region_list and len(region_list) > 0:
        filtered = filtered[filtered['Region'].isin(region_list)]
    if month_list and len(month_list) > 0:
        filtered = filtered[filtered['Migration_Start_Month'].isin(month_list)]
    if temp_range:
        filtered = filtered[
            (filtered['Temperature_C'] >= temp_range[0]) &
            (filtered['Temperature_C'] <= temp_range[1])
        ]
    if dist_range:
        filtered = filtered[
            (filtered['Flight_Distance_km'] >= dist_range[0]) &
            (filtered['Flight_Distance_km'] <= dist_range[1])
        ]
    if alt_range:
        filtered = filtered[
            (filtered['Max_Altitude_m'] >= alt_range[0]) &
            (filtered['Max_Altitude_m'] <= alt_range[1])
        ]

    return filtered

def create_map_figure(filtered_df, sample_size=100):
    """Create migration routes map"""
    df_sample = filtered_df.sample(n=min(sample_size, len(filtered_df)), random_state=42) if len(filtered_df) > sample_size else filtered_df
    
    fig = go.Figure()
    
    # Add migration routes as lines
    for idx, row in df_sample.iterrows():
        fig.add_trace(go.Scattergeo(
            lon=[row['Start_Longitude'], row['End_Longitude']],
            lat=[row['Start_Latitude'], row['End_Latitude']],
            mode='lines',
            line=dict(width=1, color='blue'),
            opacity=0.3,
            showlegend=False,
            hoverinfo='skip'
        ))
    
    # Add start points
    fig.add_trace(go.Scattergeo(
        lon=df_sample['Start_Longitude'],
        lat=df_sample['Start_Latitude'],
        mode='markers',
        marker=dict(size=5, color='green'),
        name='Start',
        hovertemplate='<b>Start</b><br>Species: %{customdata[0]}<br>Region: %{customdata[1]}<extra></extra>',
        customdata=df_sample[['Species', 'Region']].values
    ))
    
    # Add end points
    fig.add_trace(go.Scattergeo(
        lon=df_sample['End_Longitude'],
        lat=df_sample['End_Latitude'],
        mode='markers',
        marker=dict(size=5, color='red'),
        name='End',
        hovertemplate='<b>End</b><extra></extra>'
    ))
    
    fig.update_layout(
        title=f'Migration Routes (Sample of {len(df_sample)})',
        geo=dict(
            projection_type='natural earth',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            coastlinecolor='rgb(204, 204, 204)',
        ),
        height=500,
        margin=dict(l=0, r=0, t=40, b=0)
    )
    
    return fig

def create_overview_dashboard(filtered_df):
    """Create the main overview with 4 coordinated views"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Migration Intensity by Month (Q5)',
                        'Avg Distance by Species (Q3)',
                        'Temperature Distribution (Q2)',
                        'Altitude by Species (Q4)'),
        specs=[[{'type': 'scatter'}, {'type': 'bar'}],
               [{'type': 'histogram'}, {'type': 'bar'}]],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Line Chart - Migration Intensity
    monthly_starts = filtered_df['Migration_Start_Month'].value_counts().reindex(month_order, fill_value=0)
    monthly_ends = filtered_df['Migration_End_Month'].value_counts().reindex(month_order, fill_value=0)
    
    fig.add_trace(
        go.Scatter(x=month_order, y=monthly_starts.values, 
                   mode='lines+markers', name='Starts',
                   line=dict(color='rgb(59, 130, 246)', width=2),
                   marker=dict(size=8)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=month_order, y=monthly_ends.values,
                   mode='lines+markers', name='Ends',
                   line=dict(color='rgb(239, 68, 68)', width=2),
                   marker=dict(size=8)),
        row=1, col=1
    )
    
    # 2. Bar Chart - Species Distance
    species_distance = filtered_df.groupby('Species')['Flight_Distance_km'].mean().sort_values(ascending=True)
    fig.add_trace(
        go.Bar(y=species_distance.index, x=species_distance.values,
               orientation='h', marker=dict(color='rgb(16, 185, 129)'),
               name='Avg Distance', showlegend=False),
        row=1, col=2
    )
    
    # 3. Histogram - Temperature Distribution
    fig.add_trace(
        go.Histogram(x=filtered_df['Temperature_C'], nbinsx=20,
                     marker=dict(color='rgb(245, 158, 11)'),
                     name='Temperature', showlegend=False),
        row=2, col=1
    )
    
    # 4. Bar Chart - Species Altitude
    species_altitude = filtered_df.groupby('Species')['Max_Altitude_m'].mean().sort_values(ascending=True)
    fig.add_trace(
        go.Bar(y=species_altitude.index, x=species_altitude.values,
               orientation='h', marker=dict(color='rgb(139, 92, 246)'),
               name='Avg Altitude', showlegend=False),
        row=2, col=2
    )
    
    fig.update_xaxes(title_text="Month", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_xaxes(title_text="Distance (km)", row=1, col=2)
    fig.update_xaxes(title_text="Temperature (°C)", row=2, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_xaxes(title_text="Altitude (m)", row=2, col=2)
    
    fig.update_layout(
        height=700,
        showlegend=True,
        template='plotly_white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    return fig

def create_correlation_plots(filtered_df):
    """Create correlation scatter plots"""
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Temperature vs Distance (Q2)', 'Altitude vs Speed'),
        horizontal_spacing=0.1
    )
    
    # Temperature vs Distance
    fig.add_trace(
        go.Scatter(
            x=filtered_df['Temperature_C'],
            y=filtered_df['Flight_Distance_km'],
            mode='markers',
            marker=dict(color='rgb(59, 130, 246)', size=5, opacity=0.5),
            name='Temp vs Dist',
            showlegend=False,
            hovertemplate='Temp: %{x}°C<br>Distance: %{y} km<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Altitude vs Speed
    fig.add_trace(
        go.Scatter(
            x=filtered_df['Max_Altitude_m'],
            y=filtered_df['Average_Speed_kmph'],
            mode='markers',
            marker=dict(color='rgb(16, 185, 129)', size=5, opacity=0.5),
            name='Alt vs Speed',
            showlegend=False,
            hovertemplate='Altitude: %{x} m<br>Speed: %{y} km/h<extra></extra>'
        ),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_yaxes(title_text="Distance (km)", row=1, col=1)
    fig.update_xaxes(title_text="Altitude (m)", row=1, col=2)
    fig.update_yaxes(title_text="Speed (km/h)", row=1, col=2)
    
    fig.update_layout(
        height=400,
        template='plotly_white'
    )
    
    return fig

def create_rq3_altitude_chart(filtered_df):
    """RQ3: Bar chart of maximum altitudes per species."""
    if filtered_df.empty:
        fig = go.Figure()
        fig.add_annotation(text="No data for selected filters",
                           x=0.5, y=0.5, showarrow=False)
        return fig

    # Compute average max altitude per species
    species_altitude = (
        filtered_df.groupby('Species')['Max_Altitude_m']
        .mean()
        .sort_values(ascending=False)
        .reset_index()
    )

    fig = go.Figure(
        go.Bar(
            x=species_altitude["Species"],
            y=species_altitude["Max_Altitude_m"],
            marker=dict(color='orange'),
            hovertemplate="Species: %{x}<br>Avg Max Altitude: %{y} m<extra></extra>"
        )
    )

    fig.update_layout(
        title="RQ3: Highest Flying Species (Max Altitude)",
        xaxis_title="Species",
        yaxis_title="Average Max Altitude (m)",
        template="plotly_white",
        height=500
    )

    return fig

def create_violin_altitude_plot(filtered_df):
    """RQ3: Violin plot showing altitude distribution per species."""
    
    if filtered_df.empty:
        fig = go.Figure()
        fig.add_annotation(
            text="No data available for selected filters", 
            x=0.5, y=0.5, showarrow=False
        )
        return fig

    fig = go.Figure()

    fig.add_trace(go.Violin(
        x=filtered_df["Species"],
        y=filtered_df["Max_Altitude_m"],
        box_visible=True,
        meanline_visible=True,
        line_color="orange",
        fillcolor="rgba(255,165,0,0.3)",
        name="Altitude Distribution",
        spanmode="soft"
    ))

    fig.update_layout(
        title="RQ3: Altitude Distribution per Species (Violin Plot)",
        xaxis_title="Species",
        yaxis_title="Max Altitude (m)",
        template="plotly_white",
        height=550
    )

    return fig

def create_rq3_boxplot(filtered_df):
    """RQ3: Boxplot of altitude distribution per species."""
    if filtered_df.empty:
        fig = go.Figure()
        fig.add_annotation(text="No data for selected filters",
                           x=0.5, y=0.5, showarrow=False)
        return fig

    fig = go.Figure()

    fig.add_trace(go.Box(
        x=filtered_df["Species"],
        y=filtered_df["Max_Altitude_m"],
        boxmean=True,
        marker_color="teal",
        name="Altitude Distribution"
    ))

    fig.update_layout(
        title="RQ3: Boxplot of Altitude per Species",
        xaxis_title="Species",
        yaxis_title="Max Altitude (m)",
        template="plotly_white",
        height=550
    )
    return fig

def create_rq23_combined_plot(filtered_df):
    """RQ2 + RQ3 combined: Scatter plot of avg distance vs avg altitude per species."""
    if filtered_df.empty:
        fig = go.Figure()
        fig.add_annotation(
            text="No data available for selected filters",
            x=0.5, y=0.5, showarrow=False
        )
        return fig

    # Compute statistics
    species_stats = (
        filtered_df.groupby("Species")
        .agg({
            "Flight_Distance_km": "mean",
            "Max_Altitude_m": "mean"
        })
        .reset_index()
    )

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=species_stats["Flight_Distance_km"],
        y=species_stats["Max_Altitude_m"],
        mode="markers+text",
        text=species_stats["Species"],
        textposition="top center",
        marker=dict(size=12, color="purple", opacity=0.8),
        hovertemplate=(
            "Species: %{text}<br>"
            "Avg Distance: %{x:.0f} km<br>"
            "Avg Altitude: %{y:.0f} m<extra></extra>"
        )
    ))

    fig.update_layout(
        title="RQ2 + RQ3 Combined: Distance vs Altitude per Species",
        xaxis_title="Average Flight Distance (km)",
        yaxis_title="Average Max Altitude (m)",
        template="plotly_white",
        height=600
    )

    return fig

### Dash app

In [111]:
app = dash.Dash(__name__, 
                external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP],
                suppress_callback_exceptions=True)

# Get unique values for filters
unique_species = sorted(df['Species'].unique())
unique_regions = sorted(df['Region'].unique())

### App Layout


In [112]:
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1("🦅 Bird Migration Analysis Dashboard", 
                   className="text-center mb-2",
                   style={'color': '#0259b1ff', 'fontWeight': 'bold'}),
            html.P("Interactive exploration of migration patterns, routes, and environmental factors",
                   className="text-center text-muted mb-4"),
        ])
    ]),
    
    # KPI Cards
    html.H3("📊 Key Statistics and Highlights", className="mt-4 mb-3"),
    dbc.Row([
        dbc.Col(make_kpi_card("Total Observations", summary_stats['Total Observations'], "bi bi-geo-alt"), lg=2, md=4, sm=6),
        dbc.Col(make_kpi_card("% Successful", summary_stats['% of Successful Migration'], "bi bi-check-circle"), lg=2, md=4, sm=6),
        dbc.Col(make_kpi_card("Maximal Altitude", summary_stats['Maximal Altitude'], "bi bi-arrow-up-circle"), lg=2, md=4, sm=6),
        dbc.Col(make_kpi_card("Avg Flight Duration", summary_stats['Average Flight Duration'], "bi bi-clock-history"), lg=2, md=4, sm=6),
        dbc.Col(make_kpi_card("Avg Flight Distance", summary_stats['Average Flight Distance'], "bi bi-globe-americas"), lg=2, md=4, sm=6),
        dbc.Col(make_kpi_card("Longest Flight", summary_stats['Longest Flight'], "bi bi-trophy"), lg=2, md=4, sm=6),
    ], className="g-3 mb-4"),
    
    html.Hr(),
    
    # Filters Section
    html.H3("🔍 Filters & Visual Queries", className="mt-4 mb-3"),
    dbc.Card([
        dbc.CardBody([
            dbc.Row([
                # Species Filter
                dbc.Col([
                    html.Label("Species:", className="fw-bold"),
                    dcc.Dropdown(
                        id='species-filter',
                        options=[{'label': s, 'value': s} for s in unique_species],
                        value=[],
                        multi=True,
                        placeholder="Select species..."
                    ),
                ], lg=4, md=6, className="mb-3"),
                
                # Region Filter
                dbc.Col([
                    html.Label("Region:", className="fw-bold"),
                    dcc.Dropdown(
                        id='region-filter',
                        options=[{'label': r, 'value': r} for r in unique_regions],
                        value=[],
                        multi=True,
                        placeholder="Select regions..."
                    ),
                ], lg=4, md=6, className="mb-3"),
                
                # Month Filter
                dbc.Col([
                    html.Label("Migration Start Month:", className="fw-bold"),
                    dcc.Dropdown(
                        id='month-filter',
                        options=[{'label': m, 'value': m} for m in month_order],
                        value=[],
                        multi=True,
                        placeholder="Select months..."
                    ),
                ], lg=4, md=12, className="mb-3"),
            ]),
            
            dbc.Row([
                # Temperature Range
                dbc.Col([
                    html.Label("Temperature Range (°C):", className="fw-bold"),
                    dcc.RangeSlider(
                        id='temp-slider',
                        min=df['Temperature_C'].min(),
                        max=df['Temperature_C'].max(),
                        value=[df['Temperature_C'].min(), df['Temperature_C'].max()],
                        marks={int(df['Temperature_C'].min()): f"{int(df['Temperature_C'].min())}°C",
                               int(df['Temperature_C'].max()): f"{int(df['Temperature_C'].max())}°C"},
                        tooltip={"placement": "bottom", "always_visible": True}
                    ),
                ], lg=6, md=12, className="mb-3"),
                
                # Distance Range
                dbc.Col([
                    html.Label("Distance Range (km):", className="fw-bold"),
                    dcc.RangeSlider(
                        id='dist-slider',
                        min=df['Flight_Distance_km'].min(),
                        max=df['Flight_Distance_km'].max(),
                        value=[df['Flight_Distance_km'].min(), df['Flight_Distance_km'].max()],
                        marks={int(df['Flight_Distance_km'].min()): f"{int(df['Flight_Distance_km'].min())}km",
                               int(df['Flight_Distance_km'].max()): f"{int(df['Flight_Distance_km'].max())}km"},
                        tooltip={"placement": "bottom", "always_visible": True}
                    ),
                ], lg=6, md=12, className="mb-3"),
            ]),
            
                # Altitude slider directly below Distance slider
                dbc.Row([
                    dbc.Col(
                        [
                            html.Label("Altitude Range (m):", className="fw-bold"),
                            dcc.RangeSlider(
                                id='altitude-slider',
                                min=df['Max_Altitude_m'].min(),
                                max=df['Max_Altitude_m'].max(),
                                value=[df['Max_Altitude_m'].min(), df['Max_Altitude_m'].max()],
                                marks={
                                    int(df['Max_Altitude_m'].min()): f"{int(df['Max_Altitude_m'].min())}m",
                                    int(df['Max_Altitude_m'].max()): f"{int(df['Max_Altitude_m'].max())}m",
                                },
                                tooltip={"placement": "bottom", "always_visible": True}
                            )
                        ],
                        lg=6, md=12,
                        className="mb-3 offset-lg-6"   # ← THIS moves it under the distance slider
                    )
                ]),

            dbc.Row([
                dbc.Col([
                    dbc.Button("Reset All Filters", id="reset-button", color="secondary", className="mt-2"),
                    html.Span(id="filter-info", className="ms-3 text-muted")
                ])
            ])
        ])
    ], className="mb-4 shadow-sm"),
    
    html.Hr(),
    
    # Tabs for different views
    html.H3("📈 Interactive Visualizations", className="mt-4 mb-3"),
    dbc.Tabs([
        dbc.Tab(label="Overview Dashboard", tab_id="overview"),
        dbc.Tab(label="Migration Routes (Q1)", tab_id="routes"),
        dbc.Tab(label="Correlation Analysis (Q2)", tab_id="correlation"),
        dbc.Tab(label="Species Comparison (Q3 & Q4)", tab_id="species"),
        dbc.Tab(label="RQ3: Altitude Distribution", tab_id="rq3"),
        dbc.Tab(label="RQ2 + RQ3 Combined", tab_id="rq23"),
    ], id="tabs", active_tab="overview", className="mb-3"),
    
    # Content area
    html.Div(id="tab-content", className="mb-4"),
    
    # Footer
    html.Hr(),
    html.P("DSK 808: Visualization | University of Southern Denmark | Interactive Dashboard with Brushing, Filtering & Visual Queries",
           className="text-center text-muted small")
    
], fluid=True, style={'backgroundColor': '#f8f9fa'})

### CallBacks

In [113]:
@app.callback(
    [Output('species-filter', 'value'),
     Output('region-filter', 'value'),
     Output('month-filter', 'value'),
     Output('temp-slider', 'value'),
     Output('dist-slider', 'value'),
     Output('altitude-slider', 'value')],
    Input('reset-button', 'n_clicks'),
    prevent_initial_call=True
)
def reset_filters(n_clicks):
    return [], [], [], [df['Temperature_C'].min(), df['Temperature_C'].max()], [df['Flight_Distance_km'].min(), df['Flight_Distance_km'].max()]

@app.callback(
    [Output('tab-content', 'children'),
     Output('filter-info', 'children')],
    [Input('tabs', 'active_tab'),
     Input('species-filter', 'value'),
     Input('region-filter', 'value'),
     Input('month-filter', 'value'),
     Input('temp-slider', 'value'),
     Input('dist-slider', 'value'),
     Input('altitude-slider', 'value')]
)
def update_content(active_tab, species_list, region_list, month_list, temp_range, dist_range, alt_range):
    # Filter data
    filtered_df = filter_data(species_list, region_list, month_list, temp_range, dist_range, alt_range)
    
    filter_info = f"Showing {len(filtered_df):,} of {len(df):,} migrations"
    
    if active_tab == "overview":
        fig = create_overview_dashboard(filtered_df)
        return dcc.Graph(figure=fig, config={'displayModeBar': True}), filter_info
    
    elif active_tab == "routes":
        fig = create_map_figure(filtered_df, sample_size=100)
        return dcc.Graph(figure=fig, config={'displayModeBar': True}), filter_info
    
    elif active_tab == "correlation":
        fig = create_correlation_plots(filtered_df)
        return dcc.Graph(figure=fig, config={'displayModeBar': True}), filter_info
    
    elif active_tab == "species":
        species_stats = filtered_df.groupby('Species').agg({
            'Flight_Distance_km': 'mean',
            'Max_Altitude_m': 'mean'
        }).round(2).reset_index()
        
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            name='Avg Distance (km)',
            x=species_stats['Species'],
            y=species_stats['Flight_Distance_km'],
            marker=dict(color='rgb(59, 130, 246)'),
            yaxis='y'
        ))
        
        fig.add_trace(go.Bar(
            name='Avg Altitude (m)',
            x=species_stats['Species'],
            y=species_stats['Max_Altitude_m'],
            marker=dict(color='rgb(239, 68, 68)'),
            yaxis='y2'
        ))
        
        fig.update_layout(
            title='Species Performance: Distance vs Altitude (Q3 & Q4)',
            yaxis=dict(title='Distance (km)', side='left'),
            yaxis2=dict(title='Altitude (m)', overlaying='y', side='right'),
            barmode='group',
            template='plotly_white',
            height=500
        )
        
        return dcc.Graph(figure=fig, config={'displayModeBar': True}), filter_info
    
    elif active_tab == "rq3":
        # If ONE species is selected -> show only the violin plot
        if species_list and len(species_list) == 1:
            main_fig = create_violin_altitude_plot(filtered_df)
            return dcc.Graph(figure=main_fig, config={'displayModeBar': True}), filter_info

        # If NO species or MULTIPLE species -> show bar chart
        bar_fig = create_rq3_altitude_chart(filtered_df)
        return dcc.Graph(figure=bar_fig, config={'displayModeBar': True}), filter_info
    
    elif active_tab == "rq23":
        fig = create_rq23_combined_plot(filtered_df)
        return dcc.Graph(figure=fig, config={'displayModeBar': True}), filter_info

### Run App

In [114]:
if __name__ == '__main__':
    print(" Starting Bird Migration Dashboard...")
    print("Dashboard will open in your browser")
    print(" Access at: http://127.0.0.1:8050")
    app.run(debug=True, port=8050)

 Starting Bird Migration Dashboard...
Dashboard will open in your browser
 Access at: http://127.0.0.1:8050
